# Authentifizierung, Autorisierung und APIs

Neben der Session-basierten Authentifizierung, die im Abschnitt "Sessions und Cookies" vorgestellt wird, verwenden viele Webseiten auch andere Authentifizierungs- und Autorisierungsmethoden, um den Zugang zu Ressourcen zu kontrollieren, insbesondere, wenn die Ressourcen über eine API bereitgestellt werden, aber nicht allen zugänglich sein sollen. **Authentifizierung** beschreibt dabei das Verfahren, mit dem überprüft wird, ob ein Client wirklich der ist, der er vorgibt zu sein, und **Autorisierung** beschreibt die anschließende Erteilung von Zugangsrechten zu Ressourcen an den Client.


### Session- vs. Token-basierte Authentifizierung

Wir betrachten zunächst allgemein die sogenannte "Token-basierte Authentifizierung".

Damit sich ein Client gegenüber einem Server authentifizieren kann, muss er irgendwie eindeutig identifizierbar sein. Eine Möglichkeit, wie ein Server einen Client über mehrere Anfragen hinweg wiedererkennen kann, sind Sessions. Im [Kapitel 6.4](https://lipogg.github.io/webscraping-mit-python/chapters/06/subchapters/04_http.html#http) habt ihr gesehen, dass die Session-basiete Authentifizierung darauf abzielt, das zustandslose HTTP-Protokoll mithilfe einer Session um einen serverseitig gespeicherten Zustand zu ergänzen. Allerdings müssen die Session IDs aller Clients bei so einem Session-basierten Authentifizierungsvorgang auf dem Server gespeichert werden. Wenn verschiedene Clients unterschiedliche Zugriffsrechte haben, muss auch diese Information auf dem Server gespeichert sein. Dies hat den Nachteil, dass bei vielen Clients viel Speicherplatz notwendig wird. Eine in dieser Hinsicht effizientere Methode, Clients zu identifizieren und deren Zugriffsrechte zu verwalten, ist die Verwendung von Tokens. Tokens sind im Grunde Zeichenfolgen, welche die Identität oder Berechtigungen von Clients repräsentieren, und die der Server dekodieren und validieren kann. Bei der Token-basierten Authentitfizierung speichert der Server keine Sitzungsdaten, sondern der Client übermittelt bei jeder Anfrage ein Zugangstoken, das jedes Mal erneut vom Server dekodiert wird.

In der Praxis ist der Zugriff auf Ressourcen mithilfe von Zugriffstokens jedoch etwas komplizierter und beinhaltet neben der eigentlichen Authentifizierung oft auch eine Autorisierung des Clients, der die Anfrage stellt, also eine Erteilung von Zugriffsrechten auf bestimmte Ressourcen. Wie genau die Verwaltung der Zugriffsrechte erfolgt, hängt von dem spezifischen Framework oder Standard ab, der zur Implementierung des Authentifizierungs- bzw. Autorisierungsworkflows verwendet wird. Innerhalb eines Standards gibt es zudem für verschiedene Zwecke verschiedene Workflows: zum Beispiel kann es einen Workflow geben, der für die Mensch-Maschine Kommunikation vorgesehen ist (ein:e User:in möchte sich mit den Zugangsdaten von sozialem Netzwerk X in einer anderen Anwendung Y anmelden), und einen anderen Workflow, der für eine Maschine-zu-Maschine Kommunikation entworfen wurde (das ist unser Fall: Python requests wird verwendet, um Ressourcen von einem Server anzufragen). Im folgenden Abschnitt betrachten wir ein solches komplexeres Verfahren zur Verwaltung von Zugriffsrechten.

### Autorisierung mit OAuth 2.0

Ein zur Verwaltung von Zugriffsrechten auf APIs viel genutzter Standard ist **OAuth 2.0**. OAuth 2.0 steht für "Open Authorization" und ist, einfach formuliert, eine Sammlung von Regeln und Verfahren, die festlegen, wie ein Client gegenüber einem Server eine erfolgte Authentifizierung nachweisen kann, um über eine Schnittstelle Zugriff auf bestimmte Ressourcen zu erhalten (Autorisierung). Die unterschiedlichen Workflows, wie der Client für verschiedene Zwecke Zugriffsrechte erhalten kann, werden im Kontext von Oauth 2.0 "**Grants**" genannt. So gibt es beispielsweise einen "Authorization Code Grant", der für die Mensch-Maschine-Kommunikation vorgesehen ist (z.B. beim Login eine:r User:in mit den Zugangsdaten eines sozialen Netzwerks X in einer anderen Anwendung Y), und es gibt den für uns wichtigen "Client Credentials Grant Type", welcher für die nicht-interaktive Maschine-zu-Maschine-Kommunikation verwendet wird (mehr dazu könnt ihr [hier](https://auth0.com/intro-to-iam/what-is-oauth-2) nachlesen).

Bei einem Zugriffsvorgang mit OAuth 2.0 können unabhängig vom spezifischen Workflow (Grant) grundsätzlich vier Parteien beteiligt sein: Ein **Resource Owner**, ein **Client**, ein **Authorization Server** und ein **Resource Server**. Der Authorization Server hat zwei Endpunkte: Einen Autorisierungsendpunkt und einen Token-Endpunkt. Bei einem **Authorization Code Workflow** wäre der Resource Owner beispielsweise ein:e Nutzer:in des sozialen Netzwerks X. Der Client wäre die Anwendung Y, bei der sich die Nutzer:in mit dem Account aus dem sozialen Netzwerk X anmelden möchte. Damit die Anmeldung erfolgen kann, muss der Client (also die Anwendung Y) an den Autorisierungsendpunkt des Authorization Servers des sozialen Netzwerks eine Anfrage stellen, ob er auf Daten aus dem Account des Resource Owners zugreifen darf. Dies muss der Resource Owner interaktiv durch Eingabe der eigenen Zugangsdaten beim sozialen Netzwerk bestätigen, und erst dann erhält der Client vom Authorization Server des sozialen Netzwerks ein Access Token. Dieses Token kann der Client dann verwenden, um eine Anfrage an den Resource Server des sozialen Netzwerks zu stellen, auf dem die Daten des Resource Owners liegen. Da die Zugriffsrechte in dem Access Token klar definiert sind, erhält der Client vom Resource Server genau die Daten, für die der Resource Owner zuvor die Zugriffserlaubnis erteilt hat. [Dieses Video](https://youtu.be/CPbvxxslDTU?si=RcYepnb1WxWQNNe4) erläutert leicht verständlich den Authorization Code Workflow.

:::{figure-md}
<img src="oauth2_authorization_code_workflow.png" alt="Beispiel OAuth 2.0 Authorization Code Workflow" class="bg-transparent" width="75%">

OAuth 2.0 Authorization Code Workflow: Beispiel Login. Adaptiert von [Okta, Inc 2024](https://auth0.com/docs/get-started/authentication-and-authorization-flow/authorization-code-flow#how-authorization-code-flow-works)
.
:::

Bei einem **Client Credentials Workflow** wäre dagegen der Resource Owner sozusagen das soziale Netzwerk selbst, sodass die Erteilung der Zugriffsrechte keine interaktive Authentifizierung und Autorisierung durch eine:n Nutzer:in voraussetzt. Allerdings wird der Begriff "Resource Owner" im Kontext von einem Client Credentials Workflow nicht verwendet, und man kann sich den Workflow stattdessen als Kommunikation zwischen nur drei Parteien vorstellen: Der Client authentifiziert sich direkt beim Authorization Server, indem er eine Anfrage an den Token-Endpunkt des Authorization Servers des sozialen Netzwerks stellt. Dabei gibt der Client seine  "Credentials" an, also Authentifizierungsdaten, die er zuvor von den Betreiber:innen des sozialen Netzwerks selbst erhalten hat. Ist die Authentifizierung erfolgreich, erhält der Client genau wie beim Authorization Code Workflow vom Autorisierungserver ein Access Token und kann damit anschließend auf die Ressourcen auf dem Resource Server zugreifen.

Für uns ist im Rahmen dieses Seminars vor allem der Client Credentials Grant Type relevant und wir schauen uns diesen Workflow im Folgenden am Beispiel der **TikTok Research API** etwas genauer an.

Die TikTok Research API kann laut Dokumentationsseiten von "independent and academic researchers who work for a non-profit institution" zur automatisierten Abfrage von Daten zu Videos, Kommentaren und Nutzer:innenprofilen genutzt werden (siehe [TikTok 2024: About Research API](https://developers.tiktok.com/doc/about-research-api/)). Die Nutzung der API setzt eine erfolgreiche Bewerbung voraus. Erfolgreiche Bewerber:innen erhalten anschließend einen Client Key und ein Client Secret (das sind einfach Zeichenfolgen), mit denen sie Zugang zu den Video-, User-, und Comments-Endpunkten der API erhalten können ([TikTok 2024: Getting Started](https://developers.tiktok.com/doc/research-api-get-started/)). Bevor eine Anfrage an diese Endpunkte gestellt werden kann, müssen jedoch zunächst Client Key und Client Secret gegen ein Access Token ausgetauscht werden. Dazu muss eine Anfrage an den Token-Endpunkt der API gestellt werden ([TikTok 2024: Client Access Token Management](https://developers.tiktok.com/doc/client-access-token-management)). Dem Abschnitt "Endpoint" auf der Seite "Client Access Token Management" können wir entnehmen, dass dazu eine POST-Anfrage an den Token-Endpunkt des Autorisierungsservers gestellt werden muss, und die Abschnitte "Headers" und "Request Body Parameters" geben uns Auskunft darüber, welche Angaben wir beim Aufruf der requests post()-Funktion übergeben müssen. Der Abschnitt "Response struct" gibt an, welche Daten bei einer erfolgreichen Anfrage zurückgegeben werden: Die Antwort enthält demnach ein Access Token, das im Rahmen der TikTok API  "Bearer Tokens" genannt wird. Ein "Bearer Token" ist einfach eine Art von Access Token.

Besonders interessant ist für uns die Information, dass bei der Anfrage als Grant Type "client_credentials" angegeben werden soll, denn das verrät uns, dass zur Autorisierung ein Client Credentials Workflow verwendet wird. Ich habe bereits erwähnt, dass anders als beim Authorization Code Workflow die Authentifizierung beim Client Credentials Workflow direkt auf dem Autorisierungsserver erfolgt. Ein Blick in die OAuth 2.0-Dokumentationsseiten verrät uns, dass bei der TikTok Research API dazu offenbar eine Authentifizierungsmethode angewandt wird, die sich Client Secret nennt ([Okta, Inc 2024: Client Credentials Flow](https://auth0.com/docs/get-started/authentication-and-authorization-flow/client-credentials-flow) und [Application Credentials](https://auth0.com/docs/secure/application-credentials)).

Der gesamte Workflow für die Nutzung der TikTok Research API sieht dann ungefähr so aus:

:::{figure-md}
<img src="oauth2_client_credentials_workflow.png" alt="TikTok OAuth 2.0 Client Credentials Workflow" class="bg-transparent" width="75%">

OAuth 2.0 Client Credentials Workflow: Beispiel TikTok Research API. Adaptiert von [Okta, Inc 2024](https://auth0.com/docs/get-started/authentication-and-authorization-flow/client-credentials-flow#how-it-works).
:::

Der folgende Code von Kursteilnehmerin Annabella Backes zeigt, wie dieser Workflow in Python reproduziert werden kann. Zunächst müssen die Client Credentials gegen ein Access Token getauscht werden, indem eine Anfrage an den Token-Endpunkt des Authorization Servers gestellt wird:

In [ ]:
import requests

# Zugangsdaten sollten hier aber wieder eingelesen werden, s. Abschnitt 11.2
CLIENT_KEY = 'your_key'
CLIENT_SECRET = 'your_secret'

# Token endpoint
token_endpoint = 'https://open.tiktokapis.com/v2/oauth/token/'

# Parameters for the request
data = {
    'client_key': CLIENT_KEY,
    'client_secret': CLIENT_SECRET,
    'grant_type': 'client_credentials'
}

# Headers for the request
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Cache-Control': 'no-cache'
}

# Send a POST request for the token
response = requests.post(token_endpoint, data=data, headers=headers)

# Extract the token from the response
bearer_token = response.json().get('access_token', '')

Anschließend kann eine Anfrage an die Endpunkte der Research API gestellt werden, um Daten vom Resource Server abzufragen. Eine Anfrage an den User-Endpunkt sähe dann beispielsweise so aus (Code wieder von Annabella Backes):

In [ ]:
user_query_endpoint = 'https://open.tiktokapis.com/v2/research/user/info/?fields=display_name,bio_description,avatar_url,is_verified,follower_count,following_count,likes_count,video_count'

user_data = {
    'username': 'some_username'
}

headers = {
    'authorization': f'bearer {bearer_token}',
    'Content-Type': 'application/json'
}

response_user = requests.post(user_query_endpoint, json=user_data, headers=headers)
result_user = response_user.json()

Zuletzt noch ein Wort zum **Begriff Autorisierung**. Die Entwickler:innen des Standards betonen, dass OAuth 2.0 ein Autorisierungsprotokoll ist und explizit kein Authentifizierungsprotokoll. Das bedeutet, dass OAuth 2.0 hauptsächlich dazu entworfen wurde, Zugang zu Ressourcen zu gewähren, und nicht zur Überprüfung der Identität des Clients, der Zugang anfragt ([Okta, Inc. 2024](https://auth0.com/intro-to-iam/what-is-oauth-2)). Zwar ist die Authentifizierung in vielen Zugriffsszenarien ein wesentlicher Bestandteil des Autorisierungsvorgangs: Zuerst wird mithilfe eines Authentifizierungsvorgangs die Identität des anfragenden Clients bestätigt, und erst danach wird eine Autorisierung erteilt und der Zugriff auf die angefragte Ressource gewährt. OAuth 2.0 beschreibt dabei jedoch nur, wie Clients die erfolgte Authentifizierung gegenüber einem Autorisierungsserver nachweisen können, um daraufhin Zugriffsberechtigungen auf Ressourcen zu erhalten, und nicht, wie die Authentifizierung selbst erfolgen soll. Der Client Credentials Workflow stellt also in dieser Hinsicht einen Spezialfall dar, weil kein Resource Owner die Autorisierung erteilt und die Zugriffsrechte des Clients bereits in den  Credentials, die ihm von den Betreiber:innen der API zugeteilt werden, geregelt sind. Wenn im Client Credentials Workflow die Credentials gegen ein Access Token ausgetauscht werden, erfolgt also eine einfache Authentifizierung des Clients und es muss keine manuelle Autorisierung durch den Resource Owner erfolgen wie bei dem Authorization Code Workflow.

### Authentifizierung mit API-Schlüssel (Key)

Neben APIs, die ein Token-System zur Authentifizierung verwenden, gibt es auch APIs, die den Zugang zu Ressourcen über einen einfachen Zugangsschlüssel (Key) kontrollieren. Ein Beispiel ist die [API des Guardian](https://open-platform.theguardian.com/documentation/). Um die Schnittstelle zu verwenden, muss erst über ein Online-Formular ein Schlüssel beantragt werden. Mit dem Schlüssel kann dann ganz einfach eine Anfrage gestellt werden. Der Schlüssel kann der requests get()-Funktion bei der Anfrage zusammen mit weiteren Parametern als Dictionary übergeben werden. Ein [Beispiel von GitHub-User:in Dan Nguyen](https://gist.github.com/dannguyen/c9cb220093ee4c12b840):

In [ ]:
import requests

MY_API_KEY = open("creds_guardian.txt").read().strip()
API_ENDPOINT = 'http://content.guardianapis.com/search'
my_params = {
    'from-date': "",
    'to-date': "",
    'order-by': "newest",
    'show-fields': 'all',
    'page-size': 200,
    'api-key': MY_API_KEY
}

resp = requests.get(API_ENDPOINT, my_params)

Was passiert in diesem Code? Zunächst wird der API Key aus einer Plaintext-Datei eingelesen und es werden überschüssige Leerzeichen entfernt. Danach wird die URL des content-Endpunkts der API einer Variable zugewiesen. Die beiden Variablen MY_API_KEY und API_ENDPOINT sind in diesem Beispiel groß geschrieben, weil es sich dabei um Konstanten handelt (siehe dazu meine Anmerkung im [Kapitel 8.3](https://lipogg.github.io/webscraping-mit-python/chapters/08/subchapters/03_loc_api.html)). Die zusätzlichen Parameter im Dictionary my_params sind der [Dokumentationsseite zum contents-Endpunkt](https://open-platform.theguardian.com/documentation/search) entnommen und können je nach Zielsetzung variieren.

### Weitere Authentifizierungsmethoden

Neben dem Token-basierten Zugriff mit OAuth 2.0 und API-Keys gibt es noch weitere Methoden, die für den Zugriff auf REST APIs verwendet wird. Eine davon ist die sogenante Basic Authentication. Die Authentifizierung nach dem Basic Authentication-Verfahren ist von allen hier vorgestellten  Verfahren das wohl unkomplizierteste, denn für dieses Authentifizierungsverfahren kann einfach der Parameter auth der requests get()-Funktion verwendet werden:

In [ ]:
requests.get('https://example.com', auth=('user', 'password'))

Siehe dazu auch die [requests-Dokumentationsseiten](https://requests.readthedocs.io/en/latest/user/authentication/#basic-authentication).

### Quellen

```{bibliography}
   :list: enumerated
   :filter: keywords % "tokens"
```